## Libraries

In [1]:
import mysql.connector
import datetime
import random
import pandas as pd

## Database Connexion

In [2]:
mydb = mysql.connector.connect(host='localhost', user='root', password='cdareception')
mycursor = mydb.cursor()

## Database Creation (MCD/MLD/MPD)

In [3]:
customers = [
    ('Jane', 'Dupont', 'jeandupont@example.com', '0123456789'),
    ('Jane', 'Dupont', 'jeandupont@example.com', '0123456789'), 
    ('Robert', '...Lefevre', 'robertlefevre@example.com', '02-34-56-78-90'),
    ('Alice', 'Martin', 'alicemartin@example.com', '07/89/01/23/45'),
    ('David', '/Brun', 'Nan', '0345678901'),
    ('Emilie', 'David', 'emiliedavid@example.com', '0456789012'),
    ('Franck', '...Wilson', 'franck@example.com', '05 67 89 01 23'),
    ('Grace', '/Leroy', 'Grace@example.com', '0123456789'),
    ('Henri', 'Chen', 'henrichen@example.com', 'Nan'),
    ('Isabelle', 'Garcia', 'isabellegarcia@example.com', '0234567890')
]

# Function to generate a random French first name
def generate_french_first_name():
    first_names = ['Jean', 'Marie', 'Paul', 'Sophie', 'Pierre', 'Claire', 'Luc', 'Émilie', 'Antoine', 'Camille']
    return random.choice(first_names)

# Function to generate a random French last name
def generate_french_last_name():
    last_names = ['Dubois', 'Martin', 'Lefevre', 'Leroy', 'Moreau', 'Fournier', 'Garcia', 'Roux', 'Lemoine', 'Lefevre']
    return random.choice(last_names)

# Function to generate a random phone number
def generate_phone_number():
    return ''.join(random.choices('0123456789', k=10))

# Adding at least 100 new customers
for _ in range(100 - len(customers)):
    first_name = generate_french_first_name()
    last_name = generate_french_last_name()
    email = f'{first_name.lower()}.{last_name.lower()}@example.com'
    phone_number = generate_phone_number()

    customers.append((first_name, last_name, email, phone_number))

# Print the updated list of customers
for customer in customers:
    print(customer)


('Jane', 'Dupont', 'jeandupont@example.com', '0123456789')
('Jane', 'Dupont', 'jeandupont@example.com', '0123456789')
('Robert', '...Lefevre', 'robertlefevre@example.com', '02-34-56-78-90')
('Alice', 'Martin', 'alicemartin@example.com', '07/89/01/23/45')
('David', '/Brun', 'Nan', '0345678901')
('Emilie', 'David', 'emiliedavid@example.com', '0456789012')
('Franck', '...Wilson', 'franck@example.com', '05 67 89 01 23')
('Grace', '/Leroy', 'Grace@example.com', '0123456789')
('Henri', 'Chen', 'henrichen@example.com', 'Nan')
('Isabelle', 'Garcia', 'isabellegarcia@example.com', '0234567890')
('Paul', 'Lefevre', 'paul.lefevre@example.com', '2378306952')
('Jean', 'Garcia', 'jean.garcia@example.com', '4073925913')
('Camille', 'Martin', 'camille.martin@example.com', '7669576283')
('Sophie', 'Lefevre', 'sophie.lefevre@example.com', '0078229394')
('Pierre', 'Roux', 'pierre.roux@example.com', '5821896794')
('Émilie', 'Leroy', 'émilie.leroy@example.com', '7693940090')
('Sophie', 'Lefevre', 'sophie.le

In [4]:
# Define sample data for the products and customers tables
products = [('Product A', 5.00), ('Product B', 2.20), ('Product C', 7.50), ('Product D', 4.50), ('Product E', 6.00)]

In [5]:
# Create the database if it does not exist
mycursor.execute("CREATE DATABASE IF NOT EXISTS project_1")

# Use the newly created database
mycursor.execute("USE project_1")

# Create the products table
mycursor.execute("CREATE TABLE IF NOT EXISTS products (product_id INT AUTO_INCREMENT PRIMARY KEY, product_name VARCHAR(255), unit_cost DECIMAL(10, 2))")
mydb.commit()

# Insert data into the products table
mycursor.executemany("INSERT INTO products (product_name, unit_cost) VALUES (%s, %s)", products)
mydb.commit()

# Create the customers table
mycursor.execute("CREATE TABLE IF NOT EXISTS customers (customer_id INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(255), last_name VARCHAR(255), email VARCHAR(255), phone VARCHAR(20))")
mydb.commit()

# Insert data into the customers table
mycursor.executemany("INSERT INTO customers (first_name, last_name, email, phone) VALUES (%s, %s, %s, %s)", customers)
mydb.commit()

# Create the sales table with foreign keys
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS sales (
        sale_id INT AUTO_INCREMENT PRIMARY KEY,
        sale_date DATE,
        customer_id INT,
        product_id INT,
        quantity INT,
        unit_price DECIMAL(10, 2),
        total_price DECIMAL(10, 2),
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    )
""")
mydb.commit()

# Define the start and end dates for generating sales data
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 12, 31)

# Insert data into the sales table
for i in range(1000):
    sale_date = start_date + datetime.timedelta(days=random.randint(0, 364))
    customer_id = random.randint(1, len(customers))
    product_id = random.randint(1, len(products))
    quantity = random.randint(1, 10)
    unit_price = products[product_id-1][1]
    total_price = quantity * unit_price
    mycursor.execute("INSERT INTO sales (sale_date, customer_id, product_id, quantity, unit_price, total_price) VALUES (%s, %s, %s, %s, %s, %s)", (sale_date, customer_id, product_id, quantity, unit_price, total_price))

mydb.commit()

print("Database, tables, and data successfully created!")


Database, tables, and data successfully created!


## SQL Query

In [6]:
# Query to show tables
show_tables_query = "SHOW TABLES"

# Execute the query
mycursor.execute(show_tables_query)

# Fetch all the rows
tables = mycursor.fetchall()

# Create a Pandas DataFrame
df_tables = pd.DataFrame(tables, columns=["Tables"])

# Display the DataFrame
print(df_tables)


      Tables
0  customers
1   products
2      sales


In [7]:
# Execute the query
query = '''
SELECT *
FROM customers
'''

# Load results into a Pandas DataFrame
df_customers = pd.read_sql_query(query, mydb)

# Display the DataFrame
df_customers.head()

C:\Users\Catello\AppData\Local\Temp\ipykernel_13688\3445282431.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customers = pd.read_sql_query(query, mydb)


,customer_id,first_name,last_name,email,phone
0,1,Jane,Dupont,jeandupont@example.com,0123456789
1,2,Jane,Dupont,jeandupont@example.com,0123456789
2,3,Robert,...Lefevre,robertlefevre@example.com,02-34-56-78-90
3,4,Alice,Martin,alicemartin@example.com,07/89/01/23/45
4,5,David,/Brun,Nan,0345678901


In [8]:
# Execute the query
query = '''
SELECT *
FROM products
'''

# Load results into a Pandas DataFrame
df_products = pd.read_sql_query(query, mydb)

# Display the DataFrame
df_products.head()

C:\Users\Catello\AppData\Local\Temp\ipykernel_13688\2793410589.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_products = pd.read_sql_query(query, mydb)


,product_id,product_name,unit_cost
0,1,Product A,5.0
1,2,Product B,2.2
2,3,Product C,7.5
3,4,Product D,4.5
4,5,Product E,6.0


In [9]:
# Execute the query
query = '''
SELECT *
FROM sales
'''

# Load results into a Pandas DataFrame
df_sales = pd.read_sql_query(query, mydb)

# Display the DataFrame
df_sales.head()

C:\Users\Catello\AppData\Local\Temp\ipykernel_13688\3326367657.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sales = pd.read_sql_query(query, mydb)


,sale_id,sale_date,customer_id,product_id,quantity,unit_price,total_price
0,1,2023-10-10,49,4,8,4.5,36.0
1,2,2023-05-16,52,2,5,2.2,11.0
2,3,2023-03-01,100,1,10,5.0,50.0
3,4,2023-05-05,37,5,3,6.0,18.0
4,5,2023-11-29,47,1,9,5.0,45.0


In [10]:
# Close the connection
mydb.close()